In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import ultralytics
from ultralytics import YOLO

In [3]:
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.nn.tasks import DetectionModel

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from ultralytics.utils.metrics import OKS_SIGMA
from ultralytics.utils.ops import crop_mask, xywh2xyxy, xyxy2xywh
from ultralytics.utils.tal import TaskAlignedAssigner, dist2bbox, make_anchors
from ultralytics.utils.loss import BboxLoss,v8DetectionLoss

In [5]:
from ultralytics.utils import LOGGER, RANK

In [6]:
class MyCustomModelBase(DetectionModel):
    def init_criterion(self):
        return v8DetectionLoss(self)


class CustomTrainerBase(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Return a YOLO detection model."""
        model = MyCustomModelBase(cfg, nc=self.data['nc'], verbose=verbose and RANK == -1)
        if weights:
            model.load(weights)
        return model

In [7]:
trainer = CustomTrainerBase(overrides={'data': 'coco128.yaml', 'model': 'yolov8n.yaml', 'epochs': 1000})
trainer.train()
trained_model = trainer.best  # get best model

Ultralytics YOLOv8.0.199 🚀 Python-3.9.13 torch-1.12.1 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=coco128.yaml, epochs=1000, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

  0%|          | 0.00/6.66M [00:00<?, ?B/s]

Unzipping /lustre/home/user/d/drusov/datasets/coco128.zip to /lustre/home/user/d/drusov/datasets/coco128...: 100%|██████████| 263/263 [00:38<00:00,  6.92file/s]
Dataset download success ✅ (41.0s), saved to /lustre/home/user/d/drusov/datasets



  0%|          | 0.00/755k [00:00<?, ?B/s]

TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]  

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

AMP: checks passed ✅
train: Scanning /lustre/home/user/d/drusov/datasets/coco128/labels/train2017... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 301.11it/s]
train: New cache created: /lustre/home/user/d/drusov/datasets/coco128/labels/train2017.cache
val: Scanning /lustre/home/user/d/drusov/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     1/10

KeyboardInterrupt: 

In [10]:
trained_model

PosixPath('runs/detect/train/weights/best.pt')

In [7]:
models = []

In [8]:
class MyCustomModelDistillationTeacher(DetectionModel):
    def init_criterion(self):
        return v8DetectionLoss(self)

    def loss(self, batch, preds=None):
        """
        Compute loss.

        Args:
            batch (dict): Batch to compute loss on
            preds (torch.Tensor | List[torch.Tensor]): Predictions.
        """
        if not hasattr(self, 'criterion'):
            self.criterion = self.init_criterion()

        preds = self.forward(batch['img']) if preds is None else preds
        criterion = self.criterion(preds, batch)
        new_criterion = (criterion[0] - criterion[0], criterion[1])
        return new_criterion


class CustomTrainerDistillationTeacher(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Return a YOLO detection model."""
        model = MyCustomModelDistillationTeacher(cfg, nc=self.data['nc'], verbose=verbose and RANK == -1)
        if weights:
            model.load(weights)
        models.append(model)
        return None
        return model

In [9]:
trainer1 = CustomTrainerDistillationTeacher(overrides={'data': 'coco128.yaml', 'model': 'yolov8m.pt', 'epochs': 1})
trainer1.train()

Ultralytics YOLOv8.0.199 🚀 Python-3.9.13 torch-1.12.1 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=coco128.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=F

AttributeError: 'NoneType' object has no attribute 'to'

In [17]:
class MyCustomModelDistillation(DetectionModel):
    def init_criterion(self):
        self.teacher_model = models[0].cuda()

        self.mae_loss = nn.L1Loss()
        return v8DetectionLoss(self)

    def loss(self, batch, preds=None):
        """
        Compute loss.

        Args:
            batch (dict): Batch to compute loss on
            preds (torch.Tensor | List[torch.Tensor]): Predictions.
        """
        if not hasattr(self, 'criterion'):
            self.criterion = self.init_criterion()

        preds = self.forward(batch['img']) if preds is None else preds
        criterion = self.criterion(preds, batch)
        distil_part = 0.
        if len(preds) == 3:
            with torch.no_grad():
                teacher_output = self.teacher_model.forward(batch['img'])
            #print(f'TEACHER OUTPUT: {len(teacher_output)}, {teacher_output[0]}')
            distil_part = self.mae_loss(preds[0], teacher_output[0].detach()) + self.mae_loss(preds[1], teacher_output[1].detach()) + self.mae_loss(preds[2], teacher_output[2].detach())
        new_criterion = (criterion[0] + distil_part, criterion[1])
        return new_criterion


class CustomTrainerDistillation(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Return a YOLO detection model."""
        model = MyCustomModelDistillation(cfg, nc=self.data['nc'], verbose=verbose and RANK == -1)
        if weights:
            model.load(weights)
        return model

In [ ]:
trainer = CustomTrainerDistillation(overrides={'data': 'coco128.yaml', 'model': 'yolov8n.yaml', 'epochs': 1000})
trainer.train()
trained_model = trainer.best  # get best model

Ultralytics YOLOv8.0.199 🚀 Python-3.9.13 torch-1.12.1 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=coco128.yaml, epochs=1000, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

In [59]:
model_distilled = YOLO('runs/detect/train10/weights/best.pt')

In [34]:
result_distilled = model_distilled.val(data='coco128.yaml')

Ultralytics YOLOv8.0.199 🚀 Python-3.9.13 torch-1.12.1 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
YOLOv8n summary: 464 layers, 29054544 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /lustre/home/user/d/drusov/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.95it/s]
                   all        128        929      0.759      0.497      0.597       0.43
                person        128        254      0.928      0.357      0.582      0.362
               bicycle        128          6          1          0      0.171      0.112
                   car        128         46      0.613      0.109      0.143     0.0698
            motorcycle        128          5       0.88          1      0.995      0.748
              airplane        128          6      0.739      0.948      0.879      0.54

In [31]:
model_base = YOLO('runs/detect/train11/weights/best.pt')

In [36]:
result_base = model_base.val(data='coco128.yaml')

Ultralytics YOLOv8.0.199 🚀 Python-3.9.13 torch-1.12.1 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
val: Scanning /lustre/home/user/d/drusov/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.40it/s]
                   all        128        929      0.714      0.537      0.601      0.418
                person        128        254      0.859      0.409      0.599      0.391
               bicycle        128          6      0.842      0.167      0.172     0.0952
                   car        128         46      0.729       0.13      0.152     0.0862
            motorcycle        128          5      0.629          1      0.995      0.814
              airplane        128          6      0.449      0.667      0.752      0.411
                   bus        128          7       0.87      0.714      

In [39]:
result_distilled.results_dict

{'metrics/precision(B)': 0.7591659174805269,
 'metrics/recall(B)': 0.4965288827157908,
 'metrics/mAP50(B)': 0.5969456101156864,
 'metrics/mAP50-95(B)': 0.43036549784762096,
 'fitness': 0.44702350907442756}

In [41]:
result_base.results_dict

{'metrics/precision(B)': 0.7140870133225681,
 'metrics/recall(B)': 0.5373996229870989,
 'metrics/mAP50(B)': 0.6007738793443894,
 'metrics/mAP50-95(B)': 0.4183491315188963,
 'fitness': 0.4365916063014456}

In [54]:
model_distilled = YOLO('yolov8n.yaml').load('runs/detect/train10/weights/best.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [57]:
model_distilled

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s